# Short circuit analysis

Sources:

* [Short Circuit ABC](https://arcadvisor.com/files/ShortCircuitABC.pdf)
* [GridCal - Short circuit driver](https://github.com/SanPen/GridCal/blob/master/src/GridCal/Engine/Simulations/ShortCircuitStudies/short_circuit_driver.py)

## Figure 4 of [1]

![img](2023-09-xx_Figure1.png)

## MVA method

### 3 phase fault

In [1]:
from energy_tools.misc import parallel

In [2]:
system_mva = 1500
line_mva = 69**2 / 3.87
xfo_mva = 15 / 0.076
motor_mva = 15 / 0.2

upstream_mva = parallel((system_mva, line_mva, xfo_mva))
sc_mva = upstream_mva + motor_mva

sc_ka = sc_mva / (12 * 3**0.5)
print(f"3 phase fault = {sc_ka:.1f} kA ({sc_mva:.1f} MVA)")

3 phase fault = 11.0 kA (227.8 MVA)


### 1 phase fault

In [3]:
sc_neg_mva = sc_mva

# The transformer's zero sequence reactance is equal to its positive
# and negative sequence reactances
xfo0_mva = xfo_mva

# The motor's zero sequence reactance is about ½ of its positive
# sequence reactance
motor0_mva = 2 * motor_mva

sc0_mva = xfo0_mva + motor0_mva

sc0_mva = parallel((sc_mva, sc_neg_mva, sc0_mva)) * 3
sc0_ka = sc0_mva / (12 * 3**0.5)

print(f"1 phase fault = {sc0_ka:.1f} kA ({sc0_mva:.1f} MVA)")

1 phase fault = 12.4 kA (257.3 MVA)


## Pandapower

In [4]:
import pandapower as pp
import pandapower.shortcircuit as sc

Explain the IEC correction factors.

Can't explain the ~2.65 impedance factor for the transformer in single phase fault simulation. 

In [5]:
net = pp.create_empty_network(f_hz=60.0)

# Create buses
bus0 = pp.create_bus(net=net, vn_kv=69)
bus1 = pp.create_bus(net=net, vn_kv=69)
bus2 = pp.create_bus(net=net, vn_kv=12)

# Create utility
pp.create_ext_grid(
    net=net,
    bus=bus0,
    s_sc_max_mva=1500,
    rx_max=1e-20,
    rx_min=1e-20,
    x0x_max=1,  # For 1 phase fault
    r0x0_max=1,  # For 1 phase fault
)

# Create line
pp.create_line_from_parameters(
    net=net,
    from_bus=bus0,
    to_bus=bus1,
    length_km=1,
    r_ohm_per_km=1e-20,
    x_ohm_per_km=3.87 * 1.1,  # To remove IEC voltage correction factor
    c_nf_per_km=1e-20,
    r0_ohm_per_km=1e-20,
    x0_ohm_per_km=1e-20,
    c0_nf_per_km=1e-20,
    max_i_ka=1,
)

# Create transformer
xfo_3ph = pp.create_transformer_from_parameters(
    net=net,
    hv_bus=bus1,
    lv_bus=bus2,
    sn_mva=15,
    vn_hv_kv=69,
    vn_lv_kv=12,
    vkr_percent=1e-20,
    vk_percent=7.6 * 1.1,  # To remove IEC voltage correction factor
    pfe_kw=1e-20,
    i0_percent=1e-20,
    vector_group="Dyn",  # For 1 phase fault
    vkr0_percent=1e-20,  # For 1 phase fault
    vk0_percent=1e-20,  # For 1 phase fault
    mag0_percent=1e-20,  # For 1 phase fault
    si0_hv_partial=1e-20,  # For 1 phase fault
    mag0_rx=1e-20,  # For 1 phase fault
)

# Create transformer
xfo_1ph = pp.create_transformer_from_parameters(
    net=net,
    hv_bus=bus1,
    lv_bus=bus2,
    sn_mva=15,
    vn_hv_kv=69,
    vn_lv_kv=12,
    vkr_percent=1e-20,
    vk_percent=7.6 * 1.1 * 2.65,  # Trial and error
    # vk_percent=7.6 * 3,  # Almost perfect, but why???
    pfe_kw=1e-20,
    i0_percent=1e-20,
    vector_group="Dyn",  # For 1 phase fault
    vkr0_percent=1e-20,  # For 1 phase fault
    vk0_percent=1e-20,  # For 1 phase fault
    mag0_percent=1e-20,  # For 1 phase fault
    si0_hv_partial=1e-20,  # For 1 phase fault
    mag0_rx=1e-20,  # For 1 phase fault
)

# Create motor with an X1 of 0.2 pu
motor_3ph = pp.create_sgen(
    net=net,
    bus=bus2,
    p_mw=1e-20,
    sn_mva=15,
    type="motor",
    k=5,
    rx=1e-20,
)

# Create motor with an X0 of 0.1 pu
motor_1ph = pp.create_sgen(
    net=net,
    bus=bus2,
    p_mw=1e-20,
    sn_mva=15,
    type="motor",
    k=10,
    rx=1e-20,
)

buses = [
    # bus  id kv  single_phase
    (bus0, 0, 69, False),
    (bus1, 1, 69, False),
    (bus2, 2, 12, True),
]

for bus, id, kv, single_phase in buses:
    # Enable models for 3 phase fault
    net.trafo.at[xfo_3ph, "in_service"] = True
    net.sgen.at[motor_3ph, "in_service"] = True

    # Disable models for 1 phase fault
    net.trafo.at[xfo_1ph, "in_service"] = False
    net.sgen.at[motor_1ph, "in_service"] = False

    print(f"Bus {id} ({kv} kV):")
    # Run 3 phase short circuit calculation
    sc.calc_sc(net=net, bus=bus)

    sc_ka = net.res_bus_sc.loc[id, 'ikss_ka']
    sc_mva = sc_ka * kv * 3**0.5
    print(f" * 3 phase fault = {sc_ka:.1f} kA ({sc_mva:.1f} MVA)")

    if single_phase:
        # Disable models for 3 phase fault
        net.trafo.at[xfo_3ph, "in_service"] = False
        net.sgen.at[motor_3ph, "in_service"] = False

        # Enable models for 1 phase fault
        net.trafo.at[xfo_1ph, "in_service"] = True
        net.sgen.at[motor_1ph, "in_service"] = True

        # Run 1 phase short circuit calculation
        sc.calc_sc(net=net, bus=bus, fault="1ph")

        sc_ka = net.res_bus_sc.loc[id, 'ikss_ka']
        print(f" * 1 phase fault = {sc_ka:.1f} kA")

Bus 0 (69 kV):
 * 3 phase fault = 13.2 kA (1575.0 MVA)
Bus 1 (69 kV):
 * 3 phase fault = 6.3 kA (750.9 MVA)
Bus 2 (12 kV):
 * 3 phase fault = 11.0 kA (228.3 MVA)
 * 1 phase fault = 12.4 kA


## GridCal

Still can't get it to work, the documentation and code comments are not made for direct usage (without the UI).

See :

* [Five nodes tutorial](https://gridcal.readthedocs.io/en/latest/tutorials/five_node_grid.html)
* [Transformer2W](https://github.com/SanPen/GridCal/blob/master/src/GridCal/Engine/Devices/transformer.py)
* [test_short_circuit.py](https://github.com/SanPen/GridCal/blob/master/src/tests/test_short_circuit.py)

In [9]:
from GridCal.Engine import *

Bentayga is not available
Power Grid Model available
Newton Power Analytics is not available: No module named 'newtonpa'


In [10]:
np.set_printoptions(precision=4)

In [11]:
s_base_mva = 1500
grid = MultiCircuit(Sbase=s_base_mva)

# Create the 69 kV slack bus
bus1 = Bus(vnom=69)
bus1.is_slack = True
grid.add_bus(bus1)

# Create the 69 kV transformer primary bus
bus2 = Bus(vnom=69)
grid.add_bus(bus2)

# Create the 12 kV motor control center (MCC)
bus3 = Bus(vnom=12)
grid.add_bus(bus3)

# Add a 1500 MVA utility
grid.add_generator(bus1, Generator(
    Snom=s_base_mva,  # MVA
    x1=1,  # pu
))

# Add a 69 kV line
grid.add_line(Line(
    bus_from=bus1,
    bus_to=bus2,
    x=3.87 / (69**2 / s_base_mva)  # pu
))

# Add a transformer between the utility and the MCC
grid.add_transformer2w(Transformer2W(
    bus_from=bus2,
    bus_to=bus3,
    x=0.076,  # pu
    rate=15,  # MVA
))

# Add the motor (modeled as generator for short circuit)
#grid.add_generator(bus3, Generator(
#    Snom=15,  # MVA
#    x1=0.2,  # pu
#))

pf_options = PowerFlowOptions(SolverType.NR)
power_flow = PowerFlowDriver(grid, pf_options)
power_flow.run()

# Raise error if it did not converge
assert power_flow.results.converged

In [12]:
bus_index = 2

sc_options = ShortCircuitOptions(
    bus_index=[bus_index],
    fault_type=FaultType.ph3,
)

short_circuit = ShortCircuitDriver(
    grid=grid,
    options=sc_options,
    pf_options=pf_options,
    pf_results=power_flow.results,
)

short_circuit.run()

scc_mva = abs(short_circuit.results.SCpower[bus_index])
scc_ka = scc_mva / (12 * 3**0.5)
print(f"Total fault = {scc_ka:.1f} kA ({scc_mva:.1f} MVA)")

Total fault = 31.4 kA (653.5 MVA)


## PyPSA

Does not support short circuit calculations.